In [75]:
import os
import io
from hyperparams import Hyperparams as hp
import numpy as np
import tensorflow as tf
from train import Graph
from utils import *
from data_load import load_data
from scipy.io.wavfile import write
import tqdm
from IPython.display import Audio

In [18]:
import scipy
def save_wav(wav, path, sr):
  wav *= 32767 / max(0.01, np.max(np.abs(wav)))
  scipy.io.wavfile.write(path, sr, wav.astype(np.int16))

In [31]:
def load_vocab():
    char2idx = {char: idx for idx, char in enumerate(hp.vocab)}
    idx2char = {idx: char for idx, char in enumerate(hp.vocab)}
    return char2idx, idx2char

In [4]:
sess = tf.Session()

In [3]:
g = Graph(mode="synthesize"); print("Graph loaded")

Graph loaded


In [5]:
sess.run(tf.global_variables_initializer())

In [6]:
# Restore parameters
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'Text2Mel')
saver1 = tf.train.Saver(var_list=var_list)
saver1.restore(sess, tf.train.latest_checkpoint(hp.logdir + "-1"))
print("Text2Mel Restored!")

INFO:tensorflow:Restoring parameters from es/logdir-1\model_gs_400k
Text2Mel Restored!


In [7]:
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'SSRN') + \
           tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'gs')
saver2 = tf.train.Saver(var_list=var_list)
saver2.restore(sess, tf.train.latest_checkpoint(hp.logdir + "-2"))
print("SSRN Restored!")

INFO:tensorflow:Restoring parameters from es/logdir-2\model_gs_400k
SSRN Restored!


In [32]:
char2idx, idx2char = load_vocab()

In [36]:
text = 'Solo se logran estos resultados con un equipo .'.strip()+u"␃"
text

'Solo se logran estos resultados con un equipo .␃'

In [37]:
text_encode = [char2idx[char] for char in text]
print(text_encode)

[29, 51, 48, 51, 2, 55, 41, 2, 48, 51, 43, 54, 37, 50, 2, 41, 55, 56, 51, 55, 2, 54, 41, 55, 57, 48, 56, 37, 40, 51, 55, 2, 39, 51, 50, 2, 57, 50, 2, 41, 53, 57, 45, 52, 51, 2, 7, 1]


In [93]:
texts = np.zeros((1, hp.max_N), np.int32)
texts[0,:len(text)] = [char2idx[char] for char in text]
print(texts)

[[29 51 48 51  2 55 41  2 48 51 43 54 37 50  2 41 55 56 51 55  2 54 41 55
  57 48 56 37 40 51 55  2 39 51 50  2 57 50  2 41 53 57 45 52 51  2  7  1
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0 

In [39]:
len(text)

48

In [95]:
len(texts[0])

382

In [40]:
hp.max_T

522

In [44]:
hp.n_mels

80

In [54]:
L[0].shape

(382,)

In [103]:
Y = np.zeros((1,hp.max_T, hp.n_mels), np.float32)
prev_max_attentions = np.zeros((1,), np.int32)
for j in tqdm.tqdm(range(96)):
    _gs, _Y, _max_attentions, _alignments = \
        sess.run([g.global_step, g.Y, g.max_attentions, g.alignments],
                 {g.L: texts,
                  g.mels: Y,
                  g.prev_max_attentions: prev_max_attentions})
    Y[:, j, :] = _Y[:, j, :]
    prev_max_attentions = _max_attentions[:, j]



  0%|                                                   | 0/96 [00:00<?, ?it/s]

  1%|▍                                          | 1/96 [00:00<00:22,  4.31it/s]

  2%|▉                                          | 2/96 [00:00<00:19,  4.90it/s]

  3%|█▎                                         | 3/96 [00:00<00:17,  5.18it/s]

  4%|█▊                                         | 4/96 [00:00<00:17,  5.36it/s]

  5%|██▏                                        | 5/96 [00:00<00:16,  5.43it/s]

  6%|██▋                                        | 6/96 [00:01<00:16,  5.51it/s]

  7%|███▏                                       | 7/96 [00:01<00:16,  5.56it/s]

  8%|███▌                                       | 8/96 [00:01<00:15,  5.58it/s]

  9%|████                                       | 9/96 [00:01<00:15,  5.60it/s]

 10%|████▍                                     | 10/96 [00:01<00:15,  5.62it/s]

 11%|████▊                                     | 11/96 [00:01<00:15,  5.65it/s]

 12%|█████▎               

In [104]:
Z = sess.run(g.Z, {g.Y: Y})

In [105]:
Z.shape

(1, 2088, 1025)

In [106]:
Z[0,:,:].shape

(2088, 1025)

In [107]:
wav = spectrogram2wav(Z[0,:,:])

In [108]:
out = io.BytesIO()
save_wav(wav, out, hp.sr)
Audio(out.getvalue(),autoplay=True)